In [3]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as plt
from sklearn import linear_model

In [11]:
data = pd.read_csv("Pokemon 2.csv")
data.head()
# boo = True
# for index, row in data.iterrows():
#     if boo:
#         for index in range(4,10):
#             print(row[index])
#     boo = False

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [123]:
#Train the Machine Learning model with the sample data
#Create a 2D array of values detailing the average of each stat for each typing
model = pd.DataFrame()
model['Type'] = ['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying']
stats = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total_Num_Primary', 'Total_Num_Secondary']


for column in stats:
    model[column] = 0

num_per_type = data['Type 1'].value_counts()
num_per_type_2 = data['Type 2'].value_counts()

for i in num_per_type.index:
    m = model.loc[model['Type'] == i]
    model.loc[m.index, 'Total_Num_Primary'] = num_per_type[i]
    model.loc[m.index, 'Total_Num_Secondary'] = num_per_type[i]


for index1, typing in model.iterrows():
    for index2, pokemon in data.iterrows():
        if pokemon['Type 1'] == typing['Type']:
            for index in range(4,11):
                model.loc[index1, data.columns[index]] += pokemon[index]
    for col_index in range(1,8):
        model.loc[index1, model.columns[col_index]] /= model.loc[index1,'Total_Num_Primary']
#normalize values around 1 to account for variability between weak/strong pokemon
    for col_index in range(2,8):
        model.loc[index1, model.columns[col_index]] /= (model.loc[index1,'Total']/6)



model.head()

,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total_Num_Primary
0,Grass,421.142857,0.958412,1.043080,1.008684,1.104138,1.003392,0.882293,70
1,Fire,458.076923,0.915617,1.110327,0.887657,1.165491,0.945844,0.975063,52
2,Water,430.455357,1.004460,1.033582,1.016780,1.042791,0.982929,0.919458,112
3,Bug,378.927536,0.900711,1.123767,1.119865,0.852979,1.026008,0.976669,69
4,Normal,401.683673,1.154274,1.097422,0.893941,0.833736,0.951861,1.068767,98


In [111]:
pok_ind = 7
pok = data.iloc[pok_ind, 4:11]
arr = np.empty(18)
resid = 0
for col in range (1,7):
    pok[col] = (6*pok[col])/pok[0]
arr = np.empty(18)
resid = 0
for index, Type in model.iterrows():
    for col in range(1,7):
        resid += abs(pok[col]-model.iloc[index, col+1])
    if pd.to_numeric(po).idxmax() == pd.to_numeric(Type[2:8]).idxmax():
        resid -= .05
    arr[index] = resid
    resid = 0
result_ind = np.where(arr == np.amin(arr))
# print(model.loc[result_ind[0][0], 'Type'])
# print(data.iloc[pok_ind, 2])
po = pok[1:7]
print(pd.to_numeric(po).idxmax())

Attack


In [118]:
#use the predictive model to guess a pokemon's typing based on its stats alone

def predict(pok):
    #normalize the stats around 1
    for col in range (1,7):
        pok[col] = (6*pok[col])/pok[0]
    arr = np.empty(18)
    resid = 0
    for index, Type in model.iterrows():
        for col in range(1,7):
            resid += abs(pok[col]-model.iloc[index, col+1])
        arr[index] = resid
        resid = 0
    result_ind = np.where(arr == np.amin(arr))
    return(model.loc[result_ind[0][0], 'Type'])


In [119]:
count = 0
for index, pok in data.iterrows():
    guess = predict(pok[4:11])
    #or guess == pok[3]
    if guess == pok[2] :
        count+=1

hit_percent = count/len(data.index)

In [120]:
print(hit_percent)

0.14375


In [ ]:
.215
.14625